<a href="https://colab.research.google.com/github/kabilan942/Natural-Language-Processing/blob/main/Legal%20Document%20Summarization/Cosine_Manhattan_Euclidean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers==4.21.0
!pip install sentencepiece
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
!pip install rouge
nltk.download('punkt')
from rouge import Rouge

In [ ]:
import pandas as pd
import numpy as np
import glob
import json
import os
import sys
from tqdm import tqdm
sys.path.insert(0, '../')
import torch

In [ ]:
!pip install transformers==4.21.0
!pip install sentencepiece
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
!pip install rouge
nltk.download('punkt')
from rouge import Rouge
!pip install datasets
from datasets import Dataset, DatasetDict
!pip install rouge_score
!pip install datasets evaluate transformers[sentencepiece]
!pip install nltk
from nltk.tokenize import sent_tokenize
import pandas as pd
import numpy as np
import glob
import nltk
nltk.download('punkt')
from nltk import tokenize
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import torch.nn.functional as F
import torch
import json
import random
import os
import sys
sys.path.insert(0, '../')
from IPython.display import display, HTML
import torch
from datasets import load_dataset, load_metric, Dataset
from transformers import LEDTokenizer, LEDForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate
rouge_score = evaluate.load('rouge')

## Chunking - using similarity measures
1 - Cosine
2 - Manhattan
3 - Euclidean


In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=2b59eb089b259824b43a32e1e67aeea3bf53ed41de8a595af10915ec16bfdc6a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
# Loading Model and tokenizer

from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances, cosine_similarity

def similarity_l_l(l1, l2, metric):
    '''
    Function to find the most similar sentence in the document for each sentence in the summary
    input:  l1 - Summary sentences
            l2 - Document sentences
    returns a list of document sentence indexes for each sentence in the summary
    '''
    document_embeddings = sbert_model.encode(l1+l2)
    if metric==3:
      similarities=euclidean_distances(document_embeddings)
      print('Euclidean similarity')
    elif metric==2:
      similarities=manhattan_distances(document_embeddings)
      print('Manhattan similarity')
    else:
      similarities=cosine_similarity(document_embeddings)
      print('Cosine similarity')
    result = []
    for i in range(len(l1)):
        vals = similarities[i]
        vals = vals[len(l1):]
        idx = np.argmax(vals)
        result.append(idx)
    return result

In [ ]:
def nest_sentencesV2(document,chunk_length):
    '''
    function to chunk a document
    input:  document           - Input document
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a list of sentences.
    '''
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))
        if length < chunk_length:
            sent.append(sentence)  # sentences appended to sent till chunk_length is reached
        else:
            nested.append(sent)  # once reached, append sent to nested and make sent=[] and go through rest of sentences
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:  # when all sentences are over and the chunk of remaining sentences are appended to sent
        nested.append(sent)
    return nested

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
'''
def join_punctuation(seq, characters='.,;?!'):
    characters = set(characters)
    seq = iter(seq)
    current = next(seq)

    for nxt in seq:
        if nxt in characters:
            current += nxt
        else:
            yield current
            current = nxt

    yield current
'''
def get_chunks_data_from_docV2(doc, summ, m):
    '''
    Function to generate chunks along with their summaries
    input:  doc - legal Document
            summ - Gold standard summary
    returns a list of chunks and their summaries
    '''
    #text_tokens_doc = word_tokenize(doc)
    #tokens_without_sw_doc = [word for word in text_tokens_doc if not word in stopwords.words()]
    #doc = ' '.join(join_punctuation(tokens_without_sw_doc))
    #text_tokens_sum = word_tokenize(summ)
    #tokens_without_sw_sum = [word for word in text_tokens_sum if not word in stopwords.words()]
    #summ = ' '.join(join_punctuation(tokens_without_sw_sum))
    chunk_summ_word_threshold = 35
    sentence_mapping = {}
    doc_sents = split_to_sentences(doc)
    summ_sents = split_to_sentences(summ)

    result = (similarity_l_l(summ_sents,doc_sents,m)) #metric = (1,2,3) = (cosine, manhattan, euclidean)

    for i in range(len(summ_sents)):
        sentence_mapping[doc_sents[result[i]]] = summ_sents[i]

    final_chunks = []
    final_summ = []
    for chunk in nest_sentencesV2(doc, 465):
        summ = ""
        for chunk_sent in chunk:
            if chunk_sent in sentence_mapping:
                summ = summ + sentence_mapping[chunk_sent]
        if len(summ.split(" ")) >= chunk_summ_word_threshold:
            final_chunks.append(" ".join(chunk))
            final_summ.append(summ)
    return final_chunks, final_summ

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Change Path** - Training Data

In [ ]:
def get_root_path():
    '''
    function to get root path of dataset
    change the path variable to the path of the dataset
    '''
    # path = "/home/pahelibhattacharya/Abhayv2/camera_ready/summarization/dataset"
    path = "/content/drive/MyDrive/AbstractiveDataset"
    return path

def get_summary_data():
    '''
    function to get names, documents, and summaries
    change the path variable to the path of the dataset
    '''

    path = get_root_path() + '/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/summary'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f:
            a = f.read()
            l = len(a)
            data_summary.append(a)

    return names, data_source, data_summary

def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)   # returns list of sentences from para
    return sents


In [ ]:
names, data_source, data_summary = get_summary_data()
print(len(names))
print(len(data_source))
print(len(data_summary))


1
1
1


In [ ]:
import pandas as pd
from tqdm import tqdm
training_chunks = []
training_summs = []
doc_id = []
for i in tqdm(range(len(data_source))):
    cks, summs = get_chunks_data_from_docV2(data_source[i],data_summary[i],1) #metric = (1,2,3) = (cosine, manhattan, euclidean)
    training_chunks = training_chunks + cks
    training_summs = training_summs + summs
    for j in range(len(cks)):
      doc_id.append(names[i])
df = pd.DataFrame({'data':training_chunks,'summary':training_summs,'doc_id':doc_id})
df.to_excel('FD_MCS_512_3_train.xlsx')


100%|██████████| 1/1 [00:10<00:00, 10.80s/it]

Cosine similarity


In [ ]:
training_summs

['The charge created in respect of municipal property tax by section 212 of the City of Bombay Municipal Act, 1888, is an "annual charge not being a capital charge" within the mean ing of section 9 (1) (iv) of the Indian Income tax Act, 199.2, and the amount of such charge should therefore be deducted in computing the income from such property for the purposes of section 9 of the Indian Income tax Act.',
 'The charge in respect of urban immoveable property tax created by the Bombay Finance Act, 1939 is similar in character and the amount of such charge should also be deducted.554The expression "capital charge" in s.9(1) (iv) means a charge created for a capital sum,that is to say, a charge created to. \'secure the discharge of a liability of a capi tal nature; and an "annual charge" means a charge to secure an annual liabili ty.']

**Change Path** - Test dataset

In [ ]:
# for test dataset

def get_root_path():
    '''
    function to get root path of dataset
    change the path variable to the path of the dataset
    '''
    path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Single/test/"
    return path

def get_summary_data():
    '''
    function to get names, documents, and summaries
    change the path variable to the path of the dataset
    '''

    path = get_root_path() + '/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/summary'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f:
            a = f.read()
            l = len(a)
            data_summary.append(a)

    return names, data_source, data_summary


In [ ]:
names, data_source, data_summary = get_summary_data()
print(len(names))
print(len(data_source))
print(len(data_summary))

6
6
6


In [ ]:
import pandas as pd
from tqdm import tqdm
training_chunks = []
training_summs = []
doc_id = []
for i in tqdm(range(len(data_source))):
    cks, summs = get_chunks_data_from_docV2(data_source[i],data_summary[i],1) #metric = (1,2,3) = (cosine, manhattan, euclidean)
    training_chunks = training_chunks + cks
    training_summs = training_summs + summs
    for j in range(len(cks)):
      doc_id.append(names[i])
df = pd.DataFrame({'data':training_chunks,'summary':training_summs,'doc_id':doc_id})
df.to_excel('FD_MCS_512_3_test.xlsx')


 17%|█▋        | 1/6 [00:36<03:02, 36.58s/it]

Cosine similarity


 33%|███▎      | 2/6 [00:53<01:38, 24.74s/it]

Cosine similarity


 50%|█████     | 3/6 [01:26<01:25, 28.59s/it]

...

Cosine similarity


 83%|████████▎ | 5/6 [01:46<00:17, 17.26s/it]

Cosine similarity


100%|██████████| 6/6 [04:14<00:00, 42.34s/it]

Cosine similarity


## Fine-tuning T5



In [ ]:
filepath = 'FD_MCS_512_3_train.xlsx'
df = pd.read_excel(filepath,usecols='B,C')

In [ ]:
df.shape

(22, 2)

In [ ]:
df.head()

,data,summary
0,Civil Appeal No.94 1949. 107 834 Appeal judgme...,The charge created respect municipal property ...
1,"Accordingly, reasons indicated judgment, petit...",Romesh Thappar vs The State ( [ 1950 ] S.C.R.B...
2,"In construing Act, aim purpose, Act declared i...","The expression `` public safety '', result lon..."
3,XXXVII 1950. Application article 32 Constituti...,"Neither section ( 3 ) section ( 6 ) section 4,..."
4,The law providing reasonable restrictions exer...,dissenting ) ( ) unreasonable provision contai...


In [ ]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df, test_size=0.20,shuffle=False)

In [ ]:
train_df.shape,valid_df.shape

((17, 2), (5, 2))

In [ ]:
valid_df.reset_index(inplace=True)
valid_df.drop('index',inplace=True,axis=1)

In [ ]:
valid_df.head()

,data,summary
0,"By paragraph 12, village Maruthanthanallur Gna...","5,000 A, daughter N. '' Later, provisions wish..."
1,The testator mind different situations arise c...,5 intended apply case contingency village pass...
2,No. 37 1950. Appeal judgment Bombay High Court...,An application execution decree made expiry 12...
3,His darkhast attachment 856 present decree 3rd...,The decree holder contended judgment debtor ``...
4,The decree holder taking steps decree alive ci...,The decree holder appealed contending time Sup...


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(valid_df)

In [ ]:
train_dataset

Dataset({
    features: ['data', 'summary', '__index_level_0__'],
    num_rows: 17
})

In [ ]:
datasets = DatasetDict()
datasets['train'] = train_dataset.remove_columns('__index_level_0__')
datasets['valid'] = val_dataset

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['data', 'summary'],
        num_rows: 17
    })
    valid: Dataset({
        features: ['data', 'summary'],
        num_rows: 5
    })
})

In [ ]:
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
max_input_length = 512
max_target_length = 256

def preprocess_function(examples):
  # Reviews are first tokenized
  model_inputs = tokenizer(examples['data'], truncation=True, max_length=max_input_length)
  # tokenizer for the target (highlights)
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['summary'], truncation=True, max_length=max_target_length)
  # The inputs are first encoded, and then the labels ('highlights') are encoded as a separate column
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [ ]:
tokenized_dataset = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['data', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 17
    })
    valid: Dataset({
        features: ['data', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5
    })
})

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# Seq2SeqLM: sequence-to-sequence language modeling head

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")  # model_checkpoint = "t5-small"

**Change Path** - Path for model

In [ ]:
# parameters of Seq2SeqTrainingArguments
batch_size = 4
num_training_epochs = 15
logging_steps = len(tokenized_dataset["train"])//4
model_name = "t5-small-finetuned-MCS-3"
model_dir = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5"

args = Seq2SeqTrainingArguments(evaluation_strategy='steps',
                                learning_rate=5.6e-5,
                                per_device_train_batch_size=batch_size,
                                per_device_eval_batch_size=batch_size,
                                weight_decay=0.01,
                                eval_steps=2,
                                save_steps=2,
                                save_total_limit=3,
                                num_train_epochs=num_training_epochs,
                                predict_with_generate=True,
                                logging_steps=logging_steps,
                                output_dir="t5-small-finetuned-MCS-3",
                                load_best_model_at_end=True,
                                metric_for_best_model='eval_loss',
                                greater_is_better=False,
                                push_to_hub=False)


In [ ]:
import evaluate
rouge_score = evaluate.load('rouge')
def compute_metric(eval_pred):
  predictions, labels = eval_pred
  # Decode generated summaries into text (summaries generated from the review_body)
  # batch_decode -> Returns List[str]; The list of decoded sentences; Convert a list of lists of token ids into a list of strings by calling decode.
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True) # skip_special_tokens - Whether or not to remove special tokens in the decoding.
  # Replace -100 in the labels as we can't decode them
  labels = np.where(labels!=-100, labels, tokenizer.pad_token_id)
  # Decode reference summaries into text ('review_title')
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # ROUGE expects a newline after each sentence
  # strip() method removes characters from both left and right based on the argument
  decoded_preds = ['\n'.join(sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ['\n'.join(sent_tokenize(pred.strip())) for pred in decoded_labels]
  # Computing ROUGE score
  # Stemming is used to remove plurals and word suffixes such as (ing, ion, ment)
  result = rouge_score.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  result = {key: value*100 for key, value in result.items()}
  return {k: round(v,4) for k, v in result.items()}

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
datasets['train'].column_names, len(datasets['train'].column_names)

(['data', 'summary'], 2)

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(datasets['train'].column_names)
tokenized_dataset['train'].column_names

['input_ids', 'attention_mask', 'labels']

In [ ]:
from transformers import EarlyStoppingCallback
# args = Seq2SeqTrainingArguments()
# model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
# tokenizer = AutoTokenizer.from_pretrained('t5-small')
trainer = Seq2SeqTrainer(model, args, train_dataset=tokenized_dataset["train"], eval_dataset=tokenized_dataset["valid"],
                         data_collator=data_collator, tokenizer=tokenizer, compute_metrics=compute_metric,
                         callbacks=[EarlyStoppingCallback(early_stopping_patience=2)])

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 17
  Num Epochs = 15
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 75


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
2,No log,8.766941,5.337200,0.506300,5.188900,5.188900
4,7.606500,7.907529,6.464000,0.506300,5.885300,5.981000
6,7.606500,7.173917,6.139400,0.506300,5.520300,5.737600
8,5.893400,6.587154,6.139400,0.506300,5.520300,5.737600
10,5.893400,6.208655,5.450000,0.506300,4.956900,5.174200
12,5.365500,5.998305,5.871400,0.506300,5.332000,5.489100
14,5.365500,5.870716,5.871400,0.506300,5.332000,5.489100
16,4.817100,5.800640,6.326900,0.000000,5.332000,5.489100
18,4.817100,5.746162,6.326900,0.000000,5.332000,5.489100
20,4.768200,5.699948,6.326900,0.000000,5.332000,5.489100


***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
Saving model checkpoint to t5-small-finetuned-MCS-3/checkpoint-2
Configuration saved in t5-small-finetuned-MCS-3/checkpoint-2/config.json
Model weights saved in t5-small-finetuned-MCS-3/checkpoint-2/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-MCS-3/checkpoint-2/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-MCS-3/checkpoint-2/special_tokens_map.json
Copy vocab file to t5-small-finetuned-MCS-3/checkpoint-2/spiece.model
***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
Saving model checkpoint to t5-small-finetuned-MCS-3/checkpoint-4
Configuration saved in t5-small-finetuned-MCS-3/checkpoint-4/config.json
Model weights saved in t5-small-finetuned-MCS-3/checkpoint-4/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-MCS-3/checkpoint-4/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-MCS-3/checkpoint-4/special_tokens_map.jso

TrainOutput(global_step=75, training_loss=4.849761161804199, metrics={'train_runtime': 141.2415, 'train_samples_per_second': 1.805, 'train_steps_per_second': 0.531, 'total_flos': 34512159375360.0, 'train_loss': 4.849761161804199, 'epoch': 15.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 5
  Batch size = 4


{'eval_loss': 5.442809104919434,
 'eval_rouge1': 4.1502,
 'eval_rouge2': 0.0,
 'eval_rougeL': 4.4545,
 'eval_rougeLsum': 3.1846,
 'eval_runtime': 0.8866,
 'eval_samples_per_second': 5.64,
 'eval_steps_per_second': 2.256,
 'epoch': 15.0}

**Change Path** - Path of model saved in model training

In [ ]:
model_dir = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5"
trainer.save_model(model_dir)

##Validation of fine-tuned Model

In [ ]:
filepath = 'FD_MCS_512_3_test.xlsx'
test_df = pd.read_excel(filepath,usecols='B,C')

In [ ]:
def model_load(checkpoint):
     #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#Model Details
    #print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    #print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    #print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [ ]:
def sum_generate(FileContent,tokenizer,model):
    inputs = tokenizer(FileContent, return_tensors="pt", max_length=1024, truncation=True) # max_length = 1024 since length exceed otherwise and goes out of index range; genrating inputs with chunks pt stands for pytorch
    output = model.generate(inputs['input_ids'])  # inputs generated many list of tensors, from which 'input_ids' is used for generate
    summ_generated=tokenizer.decode(output[0], skip_special_tokens=True)# decoding the output  # output[0] used sinced tensor is find in [[]]
    return(summ_generated)

In [ ]:
def rouge_score_ud(summaryContent,final_summ_gen):
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    return result_rouge

In [ ]:
# inputs to the model
if __name__ == "__main__":
  checkpoint = "t5-small"
  result_rouge_list = []
  for i in range(test_df.shape[0]):
    FileContent,summaryContent=test_df['data'][i], test_df['summary'][i]
    tokenizer,model=model_load(checkpoint)
    final_summ_gen=sum_generate(FileContent,tokenizer,model)
    result_rouge_list.append(rouge_score_ud(summaryContent,final_summ_gen))

In [ ]:
result_rouge_list

In [ ]:
len(result_rouge_list)

20

In [ ]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score)

{'rouge-1': {'r': 0.23539862914862913, 'p': 0.01657821030404597, 'f': 0.029118537563018992}, 'rouge-2': {'r': 0.06696428571428571, 'p': 0.00527360066833751, 'f': 0.009755439535762474}, 'rouge-l': {'r': 0.23539862914862913, 'p': 0.01657821030404597, 'f': 0.029118537563018992}}


**Change Path** - of checkpoint to path of model

In [ ]:
# inputs to the model
if __name__ == "__main__":
  checkpoint = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5"
  result_rouge_list = []
  for i in range(test_df.shape[0]):
    FileContent,summaryContent=test_df['data'][i], test_df['summary'][i]
    tokenizer,model=model_load(checkpoint)
    final_summ_gen=sum_generate(FileContent,tokenizer,model)
    result_rouge_list.append(rouge_score_ud(summaryContent,final_summ_gen))

Didn't find file /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5/added_tokens.json. We won't load it.
loading file /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5/spiece.model
loading file /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5/tokenizer.json
loading file None
loading file /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5/special_tokens_map.json
loading file /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5/tokenizer_config.json
loading configuration file /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5/config.json
Model config T5Config {
  "_name_or_path": "/content/drive/MyDrive/

In [ ]:
result_rouge_list

[{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}},
 {'rouge-1': {'r': 0.09090909090909091,
   'p': 0.015873015873015872,
   'f': 0.027027024495982706},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.09090909090909091,
   'p': 0.015873015873015872,
   'f': 0.027027024495982706}},
 {'rouge-1': {'r': 0.5, 'p': 0.010416666666666666, 'f': 0.020408162865472725},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.5, 'p': 0.010416666666666666, 'f': 0.020408162865472725}},
 {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}},
 {'rouge-1': {'r': 0.14285714285714285,
   'p': 0.02702702702702703,
   'f': 0.04545454277892578},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.14285714285714285,
   'p': 0.02702702702702703,
   'f': 0.04545454277892578}},
 {'rouge-1': {'r': 0.0, 'p': 0.0

In [ ]:
len(result_rouge_list)

20

In [ ]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score)

{'rouge-1': {'r': 0.24427960927960932, 'p': 0.03185819425120275, 'f': 0.053347552906862425}, 'rouge-2': {'r': 0.07151806526806527, 'p': 0.007670760611937082, 'f': 0.013656083048713335}, 'rouge-l': {'r': 0.24011294261294266, 'p': 0.030695403553528337, 'f': 0.051529371088680616}}
